<a href="https://colab.research.google.com/github/osinkel/rus_voice_cloning/blob/main/Voice_Cloner_(Rus).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><font color="black" size="+4">Голосовой клонер (Русский и Английский)</font></b>

<b><font color="black" size="+2">Основан на:</font></b>

**GitHub репозитории**: [Multi-Tacotron-Voice-Cloning](https://github.com/vlomme/Multi-Tacotron-Voice-Cloning)

Автор: **[Kramarenko Vladislav](https://github.com/vlomme).**

<b><font color="black" size="+2">Colab собрал:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

```
Чтобы начать, нажмите на кнопку (куда указывает красная стрелка), после чего дождитесь завершения выполнения блока.
```
*↓ Нажмите, чтобы открыть раздел ↓*


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Клонируем репозиторий и устанавливаем все необходимые компоненты</font></b>

%cd /content
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!git clone https://github.com/vlomme/Multi-Tacotron-Voice-Cloning.git
%cd /content/Multi-Tacotron-Voice-Cloning
!pip install -r requirements.txt
!apt-get install libportaudio2

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Загружаем предобученную модель</font></b>

import os.path
from google.colab import drive
upload_type = 'Default' #@param ["Default", "Mount Disk"]

if upload_type == 'Default':
  !gdown https://drive.google.com/uc?id=1aQBmpflbX_ePUdXTSNE4CfEL9hdG2-O8
  if os.path.isfile("/content/Multi-Tacotron-Voice-Cloning/pretrained.zip"): 
    !unzip -u /content/Multi-Tacotron-Voice-Cloning/pretrained.zip
  else: 
    raise Exception("Лимиты на скачивание с Google Drive исчерпаны. Перезагрузите этот блок, предварительно поменяв upload_type на 'Mount Disk'")
else:
  print("Перейдите по следующей ссылке:")
  print("https://drive.google.com/open?id=1aQBmpflbX_ePUdXTSNE4CfEL9hdG2-O8")
  print("и нажмите 'Добавить ярлык на Диск' (кнопка в правом верхнем углу)\n")
  print("После этого перейдите по ссылке ниже, выберите аккаунт, куда сохранили предобученную модель,")
  print("нажмите на кнопку 'Разрешить', скопируйте авторизационный код в нижнее поле и нажмите Enter.")
  drive.mount('/content/drive')
  if os.path.isfile("/content/drive/My Drive/pretrained.zip"): 
    !unzip -u /content/drive/My\ Drive/pretrained.zip -d /content/Multi-Tacotron-Voice-Cloning
  else: 
    raise Exception("Что-то пошло не так. Возможно файл с предобученной моделью отсутствует или поврежден")

In [22]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Подготавливаем предобученную модель</font></b>

from IPython.display import Javascript
from google.colab import output
from google.colab import files
from base64 import b64decode
import IPython
import IPython.display as ipd
import uuid
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import warnings

!apt install --allow-change-held-packages libcudnn8=8.3.3.40-1+cuda11.5

warnings.filterwarnings("ignore")

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb+') as f:
    f.write(b)
  return 'audio.wav'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Need to get 422 MB of archives.
After this operation, 157 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.3.3.40-1+cuda11.5 [422 MB]
Fetched 422 MB in 7s (60.1 MB/s)
(Reading database ... 155660 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.3.3.40-1+cuda11.5_amd64.deb ...
Unpacking libcudnn8 (8.3.3.40-1+cuda11.5) over (8.1.0.77-1+cuda11.2) ...
Setting up libcudnn8 (8.3.3.40-1+cuda11.5) ...


In [36]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Клонируем голос</font></b>

from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

global in_fpath
global text

def del_log():
  clear_output()
  pass

def synth():
  global in_fpath
  global text

  !python demo_cli.py -p $in_fpath -t $text --no_sound
  
  if os.path.isfile("/content/Multi-Tacotron-Voice-Cloning/demo_output_00.wav"): 
    print("Если вы хотите скачать аудио, ткните правой кнопкой мыши по плееру и выберите 'Сохранить аудио'")
    ipd.display(ipd.Audio('demo_output_00.wav'))
  else: 
    raise Exception("Что-то пошло не так. Ткните 2 раза левой кнопкой мыши на этот блок. Когда код программы откроется, найдите строку 'clear_output()', удалите ее и снова запустите блок. Если эта ошибка снова возникнет, сделайте скриншот и свяжитесь с автором.")

content = 'Upload MP3, WAV or OGG file' #@param ["Upload MP3, WAV or OGG file", "Record voice"]
text = "\u041F\u043E\u0434\u043F\u0438\u0441\u044B\u0432\u0430\u0439\u0442\u0435\u0441\u044C \u043D\u0430 \u0441\u043E\u0437\u0434\u0430\u0442\u0435\u043B\u044F \u0432 \u0422\u0435\u043B\u0435\u0433\u0440\u0430\u043C\u0435" #@param {type:"string"}
text = text.replace(' ', '\ ')
#@markdown ---
seconds_of_recording = 10 #@param {type:"slider", min:10, max:30, step:2}
if content == 'Record voice':
  print("Запись в течении {} секунд началась, скажите что-нибудь...".format(str(seconds_of_recording)))
  record(seconds_of_recording)
  print("Запись голоса завершена")
  in_fpath = "audio.wav"
else:
  uploaded = files.upload().keys()
  if len(uploaded) > 1: raise Exception("Запустите этот блок снова и загрузите только ОДИН mp3, wav или ogg файл.")
  else:
    file_name = list(uploaded)[0]
    extension = file_name.split(".")[-1]
    if extension == 'wav': in_fpath = file_name
    elif extension == 'ogg' or extension == 'mp3':
      import subprocess
      process = subprocess.run(['ffmpeg', '-i', file_name, 'converted_audio.wav'])
      in_fpath = "converted_audio.wav"
    else: raise Exception("Запустите этот блок снова и загрузите один файл MP3, WAV или OGG")

del_log()
InvokeButton('Start processing', synth)

IndexError: ignored

In [35]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Повторный быстрый запуск</font></b>
#@markdown При повторном запуске может получиться более качественный результат. Если нужно подправить текст, измените его ниже:
global text
global in_fpath
warnings.filterwarnings("ignore")

new_text = "\u043F\u0440\u0438\u0432\u0435\u0442. \u0441\u0435\u0433\u043E\u0434\u043D\u044F \u0443 \u043C\u0435\u043D\u044F \u043D\u0435 \u043F\u043E\u043B\u0443\u0447\u0438\u0442\u0441\u044F \u043F\u0440\u0438\u0439\u0442\u0438 \u043D\u0430 \u0432\u0441\u0442\u0440\u0435\u0447\u0443. \u0441\u043E\u0436\u0430\u043B\u0435\u044E \u043E\u0431 \u044D\u0442\u043E\u043C" #@param {type:"string"}
if new_text == '': new_text = text
else: new_text = new_text.replace(' ', '\ ')

!python demo_cli.py -p $in_fpath -t $new_text --no_sound
clear_output()
ipd.display(ipd.Audio('demo_output_00.wav'))



---



# <b><font color="black" size="+4">Голосовой клонер (только Английский)</font></b>

<b><font color="black" size="+2">Основан на:</font></b>

**GitHub репозитории**: [Real-Time-Voice-Cloning](https://github.com/CorentinJ/Real-Time-Voice-Cloning)

Статья: [Transfer Learning from Speaker Verification to Multispeaker Text-To-Speech Synthesis](https://arxiv.org/pdf/1806.04558.pdf)

Автор: **[Corentin Jemine](https://github.com/CorentinJ).**

<b><font color="black" size="+2">Colab собрал:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

```
Чтобы начать, нажмите на кнопку (куда указывает красная стрелка), после чего дождитесь завершения выполнения блока.
```
*↓ Нажмите, чтобы открыть раздел ↓*


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Клонируем репозиторий и устанавливаем все необходимые компоненты</font></b>

%cd /content
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git
%cd /content/Real-Time-Voice-Cloning
%tensorflow_version 1.x
!pip install -q -r requirements.txt
!pip install webrtcvad
!apt-get install -qq libportaudio2

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Загружаем предобученную модель</font></b>

import os.path
from google.colab import drive
upload_type = 'Default' #@param ["Default", "Mount Disk"]

if upload_type == 'Default':
  !gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
  if os.path.isfile("/content/Real-Time-Voice-Cloning/pretrained.zip"): 
    !unzip -u /content/Real-Time-Voice-Cloning/pretrained.zip
  else: 
    raise Exception("Лимиты на скачивание с Google Drive исчерпаны. Перезагрузите этот блок, предварительно поменяв upload_type на 'Mount Disk'")
else:
  print("Перейдите по следующей ссылке:")
  print("https://drive.google.com/open?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc")
  print("и нажмите 'Добавить ярлык на Диск' (кнопка в правом верхнем углу)\n")
  print("После этого перейдите по ссылке ниже, выберите аккаунт, куда сохранили предобученную модель,")
  print("нажмите на кнопку 'Разрешить', скопируйте авторизационный код в нижнее поле и нажмите Enter.")
  drive.mount('/content/drive')
  if os.path.isfile("/content/drive/My Drive/pretrained.zip"): 
    !unzip -u /content/drive/My\ Drive/pretrained.zip -d /content/Real-Time-Voice-Cloning
  else: 
    raise Exception("Что-то пошло не так. Возможно файл с предобученной моделью отсутствует или поврежден")

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Подготавливаем предобученную модель</font></b>

from IPython.display import Javascript
from google.colab import output
from google.colab import files
from base64 import b64decode
import IPython
import IPython.display as ipd
import uuid
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb+') as f:
    f.write(b)
  return 'audio.wav'

encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Клонируем голос</font></b>

from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

global in_fpath
global text

def synth():
  global in_fpath
  global text

  reprocessed_wav = encoder.preprocess_wav(in_fpath)
  original_wav, sampling_rate = librosa.load(in_fpath)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  print("Синтезируется новое аудио...")
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  
  print("\nЕсли вы хотите скачать аудио, ткните правой кнопкой мыши по плееру и выберите 'Сохранить аудио'")
  display(Audio(generated_wav, rate=synthesizer.sample_rate))

content = 'Record voice' #@param ["Upload MP3, WAV or OGG file", "Record voice"]
text = "Subscribe to creator of this Colab on Twitter" #@param {type:"string"}
#@markdown ---
seconds_of_recording = 10 #@param {type:"slider", min:10, max:30, step:2}
if content == 'Record voice':
  print("Запись в течении {} секунд началась, скажите что-нибудь...".format(str(seconds_of_recording)))
  record(seconds_of_recording)
  print("Запись голоса завершена")
  in_fpath = Path("audio.wav")
else:
  uploaded = files.upload().keys()
  if len(uploaded) > 1: raise Exception("Запустите этот блок снова и загрузите только ОДИН mp3, wav или ogg файл")
  else:
    file_name = list(uploaded)[0]
    extension = file_name.split(".")[-1]
    if extension == 'wav': in_fpath = Path(file_name)
    elif extension == 'ogg' or extension == 'mp3':
      import subprocess
      process = subprocess.run(['ffmpeg', '-i', file_name, 'converted_audio.wav'])
      in_fpath = Path("converted_audio.wav")
    else: raise Exception("Запустите этот блок снова и загрузите один файл MP3, WAV или OGG.")

clear_output()
InvokeButton('Start processing', synth)

Synthesizing new audio...
{| ████████████████ 76000/76800 | Batch Size: 8 | Gen Rate: 6.6kHz | }
If you want to download audio, right-click on the player and select 'Save Audio'


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Повторный быстрый запуск</font></b>
#@markdown При повторном запуске может получиться более качественный результат. Если нужно подправить текст, измените его ниже:
global text
global in_fpath
warnings.filterwarnings("ignore")

new_text = "" #@param {type:"string"}
if new_text != '': text = new_text
InvokeButton('Start processing', synth)